In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv(r"/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv")

In [ ]:
df.head()

Taking a look at the data

Filling Nan values with 0

In [ ]:
df=df.fillna(0)

Checking the datatypes of each feature in the dataframe

In [ ]:
df.dtypes

Finding unique values of each feature with Object datatype :

In [ ]:
df['gender'].unique()

In [ ]:
df['ssc_b'].unique()

In [ ]:
df['hsc_b'].unique()

In [ ]:
df['hsc_s'].unique()

In [ ]:
df['degree_t'].unique()

In [ ]:
df['workex'].unique()

In [ ]:
df['specialisation'].unique()

In [ ]:
df['status'].unique()

In [ ]:
df=df.drop(['sl_no'],axis=1)

Creating a numpy array with the dataframe's values

In [ ]:
X=df.values

In [ ]:
from sklearn import preprocessing

Converting all non-numerical data into numerical values

In [ ]:
sex=preprocessing.LabelEncoder()
sex.fit(['M', 'F'])
X[:,0]=sex.transform(X[:,0])

SSLC=preprocessing.LabelEncoder()
SSLC.fit(['Others', 'Central'])
X[:,2]=SSLC.transform(X[:,2])

PUboard=preprocessing.LabelEncoder()
PUboard.fit(['Others', 'Central'])
X[:,4]=PUboard.transform(X[:,4])

PUsyl=preprocessing.LabelEncoder()
PUsyl.fit(['Commerce', 'Science', 'Arts'])
X[:,5]=PUsyl.transform(X[:,5])

Deg=preprocessing.LabelEncoder()
Deg.fit(['Sci&Tech', 'Comm&Mgmt', 'Others'])
X[:,7]=Deg.transform(X[:,7])

WorkEx=preprocessing.LabelEncoder()
WorkEx.fit(['No', 'Yes'])
X[:,8]=WorkEx.transform(X[:,8])

MBASpec=preprocessing.LabelEncoder()
MBASpec.fit(['Mkt&HR', 'Mkt&Fin'])
X[:,10]=MBASpec.transform(X[:,10])

Status=preprocessing.LabelEncoder()
Status.fit(['Placed', 'Not Placed'])
X[:,12]=Status.transform(X[:,12])

Making sure all values are of Float

In [ ]:
X=np.array(X, dtype=[('O', np.float)]).astype(np.float)
X.dtype

Multiple Linear Regression to find which factors influence a candidate in getting placed.(we take 'status' feature to train the model)
Solution to the first question asked :-

In [ ]:
xmlr=X[:,0:13]
ymlr=X[:,13]


In [ ]:
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

Splitting the data into train and test set(90% train, 10% test)

In [ ]:
xlrtrain,xlrtest,ylrtrain,ylrtest=train_test_split(xmlr,ymlr,test_size=0.1,random_state=3)
print(xlrtrain.shape,ylrtrain.shape,xlrtest.shape,ylrtest.shape)

Fitting our model:-

In [ ]:
MLR=linear_model.LinearRegression()
MLR.fit(xlrtrain,ylrtrain)

Printing all the coefficients and intercept:-

In [ ]:
print("Theta 0 :",MLR.intercept_)
print("Thetas :",MLR.coef_)

Finding importance of each feature
Note: Negative value doesnt indicate lower importance, the magnitude signifies the importance

In [ ]:
importance = MLR.coef_
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))


Plotting the feature and its coefficients to signify the importance

In [ ]:
from matplotlib import pyplot
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
yhatlr=MLR.predict(xlrtest)
print("R2 score %.2f" %r2_score(yhatlr,ylrtest))

Excluding 'status' feature and training the model :-

In [ ]:
xmlr2=X[:,0:12]
xmlr2[0:1]

ymlr2=X[:,13]
ymlr2[0:3]
xlrtrain2,xlrtest2,ylrtrain2,ylrtest2=train_test_split(xmlr2,ymlr2,test_size=0.1,random_state=3)

print(xlrtrain2.shape,ylrtrain2.shape,xlrtest2.shape,ylrtest2.shape)

MLR2=linear_model.LinearRegression()
MLR2.fit(xlrtrain2,ylrtrain2)

print("Theta 0 :",MLR2.intercept_)
print("Thetas :",MLR2.coef_)

importance2 = MLR2.coef_
print(importance2.shape)
for i,v in enumerate(importance2):
    print('Feature: %0d, Score: %.5f' % (i,v))


from matplotlib import pyplot
pyplot.bar([x for x in range(len(importance2))], importance2)
pyplot.show()



We can see that the factors/features that influence a candidate getting placed are mainly : Workex, Specialization, Gender, Degree course etc.

Applying SVM to classify students as to whether getting placed and also to find if percentage matters(Answer to question 2):-

In [ ]:
from sklearn import svm

In [ ]:
xsvm=X[:,0:12]
ysvm=X[:,12]

Splitting the data into train and test set:-

In [ ]:
xsvmtrain,xsvmtest,ysvmtrain,ysvmtest=train_test_split(xsvm,ysvm,test_size=0.1,random_state=3)


Fitting the SVM Model

In [ ]:
svmM=svm.SVC(kernel='linear')
svmM.fit(xsvmtrain,ysvmtrain)

Finding accuracy of the model :-

In [ ]:
from sklearn.metrics import jaccard_score
yhatsvm=svmM.predict(xsvmtest)
print(jaccard_score(ysvmtest,yhatsvm))

Finding factors that influence if a candidate will get placed:-

In [ ]:
print(svmM.coef_,svmM.coef_.shape)

importancesvm = svmM.coef_.transpose()
for i,v in enumerate(importancesvm):
    print('Feature: %0d, Score: %.5f' % (i,v))

importancesvm2=[0.87030,0.14679,-0.16534, 0.04070, 0.20813,-0.07558, 0.11830, -0.56383, 1.23753, -0.01899, -0.02371, -0.13834]
from matplotlib import pyplot
pyplot.bar([x for x in range(len(importancesvm2))], importancesvm2)
pyplot.show()

We can see the main factors are Workex, specialization, degree course, gender etc

Now, removing all the percentage features to see how it affects the model:-

In [ ]:
X2=X
print(X2[0:1])
X2=np.delete(X2, [1,3,6,11], axis=1)
print(X2[0:1])


In [ ]:
xsvm2=X2[:,0:8]
ysvm2=X2[:,8]


xsvmtrain2,xsvmtest2,ysvmtrain2,ysvmtest2=train_test_split(xsvm2,ysvm2,test_size=0.1,random_state=3)

svmM2=svm.SVC(kernel='linear')
svmM2.fit(xsvmtrain2,ysvmtrain2)
yhatsvm2=svmM2.predict(xsvmtest2)


importancesvm2 = svmM2.coef_.transpose()
for i,v in enumerate(importancesvm2):
    print('Feature: %0d, Score: %.5f' % (i,v))


print("Jaccard similarity without taking any percentages" ,jaccard_score(ysvmtest2,yhatsvm2))
print("Jaccard similarity with percentages" ,jaccard_score(ysvmtest,yhatsvm))

Hence, the score worsens without taking the percentages. The percentages do matter.
Thanks, hope my solution to the questions asked is helpful